In [1]:
# Install required packages (use pip install if not installed)
# pip install geopandas pykrige rasterio numpy pandas

import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging
from sklearn.metrics import mean_squared_error

# Load the CSV file into a dataframe
data = pd.read_csv("~/University/uhi/zaragoza/data.csv")

# Check the first few rows of the dataset
print(data.head())

# Create a GeoDataFrame with spatial information
points = gpd.GeoDataFrame(
    data, 
    geometry=gpd.points_from_xy(data.lon, data.lat), 
    crs="EPSG:4326"  # WGS84
)

# Define variogram parameters (similar to R's vgm)
variogram_params = {"sill": 1, "range": 1000, "nugget": 0.1}

# Perform cross-validation
folds = 10
points['fold'] = np.random.randint(0, folds, data.shape[0])  # Randomly assign folds

residuals = []

for fold in range(folds):
    train = points[points['fold'] != fold]
    test = points[points['fold'] == fold]

    # Extract train/test variables
    X_train = train[["svf", "imd", "ndvi", "swir2"]].values
    y_train = train["temp_diff"].values
    X_test = test[["svf", "imd", "ndvi", "swir2"]].values
    y_test = test["temp_diff"].values

    # Fit Kriging model
    ok = OrdinaryKriging(
        train.geometry.x, 
        train.geometry.y, 
        y_train, 
        variogram_model="spherical",
        variogram_parameters=[variogram_params['sill'], variogram_params['range'], variogram_params['nugget']]
    )
    
    # Predict on test set
    z_pred, _ = ok.execute("points", test.geometry.x, test.geometry.y)
    residuals.extend(y_test - z_pred)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error([0] * len(residuals), residuals))
print(f"RMSE: {rmse:.2f}")

# --- INTERPOLATION ---
# Paths to the .tif files
svf_path = "../data/rasters/Zaragoza_ETRS89_Sky_View_Factor_scaled.tif"
imd_path = "../data/rasters/Zaragoza_ETRS89_Imperviousness_Density_normalized_scaled.tif"
ndvi_path = "../data/rasters/Zaragoza_ETRS89_NDVI_scaled.tif"
swir2_path = "../data/rasters/Zaragoza_ETRS89_SWIR2_normalized_scaled.tif"

# Read raster layers
with rasterio.open(svf_path) as src:
    svf_raster = src.read(1)
    svf_transform = src.transform
    svf_crs = src.crs

with rasterio.open(imd_path) as src:
    imd_raster = src.read(1)

with rasterio.open(ndvi_path) as src:
    ndvi_raster = src.read(1)

with rasterio.open(swir2_path) as src:
    swir2_raster = src.read(1)

# Ensure rasters have the same resolution and extent
from rasterio.enums import Resampling
from rasterio.warp import reproject

template = svf_raster
imd_raster = rasterio.reproject(
    source=imd_raster,
    destination=np.zeros_like(template),
    src_transform=src.transform,
    dst_transform=svf_transform,
    resampling=Resampling.bilinear,
)

# Perform kriging interpolation
ok = OrdinaryKriging(
    points.geometry.x, 
    points.geometry.y, 
    points["temp_diff"], 
    variogram_model="spherical",
    variogram_parameters=[variogram_params['sill'], variogram_params['range'], variogram_params['nugget']]
)

# Create grid for interpolation
grid_x, grid_y = np.meshgrid(
    np.arange(svf_transform[2], svf_transform[2] + template.shape[1] * svf_transform[0], svf_transform[0]),
    np.arange(svf_transform[5], svf_transform[5] + template.shape[0] * svf_transform[4], svf_transform[4])
)

z_pred, _ = ok.execute("grid", grid_x, grid_y)

# Save as GeoTIFF
output_path = "~/University/uhi/zaragoza/interpolation_SVF+IMD+NDVI.tif"
with rasterio.open(
    output_path, 
    "w", 
    driver="GTiff", 
    height=z_pred.shape[0], 
    width=z_pred.shape[1], 
    count=1, 
    dtype=str(z_pred.dtype), 
    crs=svf_crs, 
    transform=svf_transform,
) as dst:
    dst.write(z_pred, 1)

print(f"Interpolated raster saved at: {output_path}")

   station       svf      ndvi  imd  swir2  temp_diff        lat       lon
0        1  0.846217  0.485734  0.0    0.0   1.396381  41.653858 -0.876258
1        2  0.889060  0.178041  0.0    0.0   1.320874  41.649671 -0.884855
2        3  0.963841  0.144677  0.0    0.0   1.069900  41.660269 -0.871698
3        4  0.906010  0.215104  0.0    0.0   1.115054  41.646083 -0.900292
4        5  0.349635  0.404729  0.0    0.0   1.633224  41.650508 -0.904937
RMSE: 0.36


AttributeError: module 'rasterio' has no attribute 'reproject'